# 5DGAI - Gemini Student Services Agent with Model Context Protocol Tooling

Author: Eidan J. Rosado

GitHub and HuggingFace Handle: `EdyVision`

This notebook was created as part of the Google Gemini + Kaggle 2025 5-day training. It leverages a HuggingFace-hosted MCP server to provide tooling for Gemini to provide rich registrar and financial aid chat interactions and summarizations. Towards the end of the project, a custom Agent is built to combine functionality into 1 seamless call and the agent's response is eventually evaluated by a Gemini AI response evaluator.

## Use Case
For my use case, I went with the standard case for many students at a college or university, the student services portal! I wanted to be able to interact with Gemini using context from student profiles and the financial aid records. When pulling the student profiles, I wanted it to maintain their course history, program details, GPA, etc. The following is a sample fictitious record for a student:

```json
{
    "student_id": "5dbeccea-947a-4817-b3cb-2c9231bac4f5",
    "name": "Valerie Gray",
    "courses": "[\"PSY110\", \"ENG202\", \"PHIL101\", \"CHEM121\"]",
    "is_need_based_qualified": true,
    "enrollment_status": "enrolled",
    "major": "Nursing",
    "program": "RN to BSN Track",
    "year": "Junior",
    "gpa": 3.27
}
```

I wanted to focus on just one thing in my interactions with Gemini…

```What financial aid options does a student qualify for?```

### Capabilities Demonstrated
- Structured Output via JSON config ✅
- Few-shot prompting ✅
- Function Calling with Gemini tool use ✅
- Gemini Agent Definition and use ✅
- AI Evaluation ✅

## Blog
<a href="https://medium.com/@docejr/experimenting-with-a-demo-mcp-and-gemini-e5b183e3b3f3">Medium Blog Post</a>

## Disclaimers
<em>The dataset behind the MCP is entirely fictitious. No real student records were used. This notebook and associated MCP are both free from any conflict of interest to the best of the author's knowledge.</em>

## Install Dependencies

In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"
! pip install mcp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-lsp 3.10.2 requires jupyterlab<4.0.0a0,>=3.1.0, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.9 MB/s eta 0:00:00


## Handle Imports and Constants

In [2]:
import numpy as np
import pandas as pd
import os
import enum
import requests
import json
import random
import asyncio
import nest_asyncio
from typing import Dict, Any, List, Optional
from pprint import pprint
from google.generativeai import GenerativeModel, configure
import google.generativeai as genai
from google import genai
from google.genai import types
from google.api_core import retry
from google.generativeai.types import (
    Tool, 
    FunctionDeclaration, 
    GenerationConfig
)

from contextlib import AsyncExitStack
from mcp import ClientSession
from mcp.client.sse import sse_client
from kaggle_secrets import UserSecretsClient


genai.__version__


'1.7.0'

In [3]:
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
HF_TOKEN = UserSecretsClient().get_secret("HF_SPACE_TOKEN")

genai_client = genai.Client(api_key=GOOGLE_API_KEY)

## Define Custom MCP Client

Now, I could have done a simple API and moved along, but I wanted to use this opportunity to get acquainted with building out a Model Context Protocol (MCP). This is similar to APIs, except AI Agents are able to interact with the exposed tools (think of these as function calls). So I ended up building the `student-services-demo-mcp` project to help me get this project going. I used the MCP to provide Gemini with context from fictitious data to generate the responses we are after.

In [4]:
class StudentServicesMCPClient:
    def __init__(self, base_url: str = "http://localhost:7860/mcp", auth_token: str = ""):
        self.session: Optional[ClientSession] = None
        self.exit_stack = AsyncExitStack()
        self.base_url = base_url
        self.base_headers = {
            "Accept": "text/event-stream",
            "Cache-Control": "no-cache",
            "Connection": "keep-alive",
            "Authorization": f"Bearer {auth_token}"
        }

    def _get_auth_headers(self) -> Dict[str, str]:
        """Get headers including authentication if token is available"""
        return self.base_headers

    async def connect_to_server(self):
        """Connect to the Hugging Face Space MCP server"""
        try:
            print(f"\nConnecting to server at: {self.base_url}")
            
            # Initialize SSE transport with headers
            try:
                sse_transport = await self.exit_stack.enter_async_context(
                    sse_client(
                        url=self.base_url,
                        headers=self._get_auth_headers(),
                    )
                )
            except Exception as e:
                print(f"Error creating SSE transport: {str(e)}")
                raise

            # Create client session with the SSE transport
            try:
                read_stream, write_stream = sse_transport
                self.session = await self.exit_stack.enter_async_context(
                    ClientSession(read_stream, write_stream)
                )
            except Exception as e:
                print(f"Error creating client session: {str(e)}")
                raise

            try:
                await self.session.initialize()
            except Exception as e:
                print(f"Error initializing session: {str(e)}")
                raise

            # List available tools
            try:
                response = await self.session.list_tools()
                tools = response.tools
                print("\nConnected to server with tools:", [tool.name for tool in tools])
            except Exception as e:
                print(f"Error listing tools: {str(e)}")
                raise

        except Exception as e:
            print(f"Error connecting to server: {str(e)}")
            raise

    async def fetch_students(self, limit: int = 100):
        """Fetch a list of students"""
        response = await self.session.call_tool("fetch_students", {"limit": limit})
        return response.content

    async def check_financial_aid_eligibility(self, student_id: str):
        """Check financial aid eligibility for a student"""
        response = await self.session.call_tool(
            "check_financial_aid_eligibility", {"student_id": student_id}
        )
        return response.content

    async def fetch_student_profile(self, student_id: str):
        """Fetch a student's profile"""
        response = await self.session.call_tool(
            "fetch_student_profile", {"student_id": student_id}
        )
        return response.content

    async def fetch_academic_history(self, student_id: str):
        """Fetch a student's academic history"""
        response = await self.session.call_tool(
            "fetch_academic_history", {"student_id": student_id}
        )
        return response.content

    async def cleanup(self):
        """Clean up resources"""
        await self.exit_stack.aclose()


In [5]:
# Connect to MCP Server
HF_BASE_URL = "https://edyvision-student-services-demo-mcp.hf.space/mcp"  # Space is private
mcp_client = StudentServicesMCPClient(base_url=HF_BASE_URL, auth_token=HF_TOKEN)
await mcp_client.connect_to_server()


Connecting to server at: https://edyvision-student-services-demo-mcp.hf.space/mcp

Connected to server with tools: ['root__get', 'fetch_student_profile', 'fetch_students', 'fetch_academic_history', 'check_financial_aid_eligibility']


### Preview Fictitious Student Records

In [6]:
limit = 10
students_result = await mcp_client.fetch_students(limit=limit)

students = json.loads(students_result[0].text)
students_df = pd.DataFrame(students)
students_df.head(5)

,student_id,name,courses,is_need_based_qualified,enrollment_status,major,program,year,gpa
0,df62674f-5641-4657-a614-901a22ea76f2,Allison Hill,"['ENG202', 'BIO204', 'PHIL101', 'CS101', 'HIST...",True,enrolled,English Literature,B.A. in Comparative Literature,Senior,2.22
1,d777f2b6-906f-4703-a5f3-efac47766ac0,Megan Mcclain,"['ENG202', 'BIO204', 'HIST300']",True,enrolled,Nursing,B.S. in Nursing (BSN),Junior,2.84
2,1daf22b9-fbdc-48d8-ac64-abb8d4b775b0,Allen Robinson,"['ENG202', 'CHEM121', 'HIST300', 'PSY110', 'CS...",False,enrolled,Nursing,B.S. in Nursing (BSN),Junior,2.05
3,c0ad1251-fef1-4ec0-8914-046fb4119492,Cristian Santos,"['ENG202', 'PHIL101', 'PSY110', 'CS101']",True,enrolled,English Literature,B.A. in Creative Writing,Senior,2.85
4,e73992d2-6535-43d8-9c3e-afb3bb6463c8,Kevin Pacheco,"['HIST300', 'PHIL101', 'CS101', 'ENG202', 'MAT...",True,enrolled,Computer Science,B.S. in Cybersecurity,Senior,2.76


Preview single record...

In [7]:
students[8]

{'student_id': '5dbeccea-947a-4817-b3cb-2c9231bac4f5',
 'name': 'Valerie Gray',
 'courses': "['PSY110', 'ENG202', 'PHIL101', 'CHEM121']",
 'is_need_based_qualified': False,
 'enrollment_status': 'enrolled',
 'major': 'Nursing',
 'program': 'RN to BSN Track',
 'year': 'Senior',
 'gpa': 3.27}

## Preview Models Available

In [8]:
for model in genai_client.models.list():
    if "createTunedModel" in model.supported_actions:
        print(model.name)

models/gemini-1.5-flash-001-tuning


## Few Shot Examples and Structured Output

In [9]:
few_shot_prompt = """Parse a student's financial aid eligibility into valid JSON:

EXAMPLE:
Student Linda Bailey (ID fed2a7af-096d-4c92-b271-911d93117d42) is a Computer Science major 
with a GPA of 3.7 and an income of $32000. They are eligible for the STEM Excellence award 
and the Income Based Grant.
JSON Response:
```
{
"name": "Linda Bailey",
"major": "Computer Science",
"gpa": 3.7,
"income": 32000,
"eligible_aid": ["STEM Excellence Award", "Income Based Grant"]
}
```

EXAMPLE:
Student John Doe (ID fed2a7af-096d-4c92-b271-911d93117d42) is a Nursing major with a GPA of 
3.32 and an income of $34000. They are eligible for the income based grant.
JSON Response:
```
{
"name": "John Doe",
"major": "Nursing",
"gpa": 3.32,
"income": 34000,
"eligible_aid": ["Income Based Grant"]
}
```

ELIGIBILITY:
"""

student_eligibility_results = "Student Tiny Tim (ID fed2a7af-096d-4c92-b271-911d93117d42) is " + \
                            "a Psychology major with a GPA of 2.5 and an income of $17500. " + \
                            "They are not eligible for any awards or grants."

response = genai_client.models.generate_content(
    model='models/gemini-1.5-flash-001-tuning',
    config=types.GenerateContentConfig(
        temperature=0.1,
        response_mime_type="application/json",
    ),
    contents=[few_shot_prompt, student_eligibility_results])

print(response.text)

{"name": "Tiny Tim", "major": "Psychology", "gpa": 2.5, "income": 17500, "eligible_aid": []}


## Gemini + Function Calling with MCP Tools

In this section, the functionality to check financial aid eligibility is performed. Using the Gemini calls outright, tools will need to be synchronous when passed in to avoid an error. the check_eligibility wrapper is created to deal with the MCP client's asynchronous calls.

In [10]:
def check_eligibility(student_id: str) -> dict:
    """
    Wraps the check_eligibility MCP tool call.
    """
    try:
        # nest_asyncio required due to notebook already running inside an event loop
        nest_asyncio.apply()  

        # wrap client call in asyncio.run()
        response =  asyncio.run(mcp_client.check_financial_aid_eligibility(student_id))
        return response[0].text
    except Exception as e:
        return {
            "error": str(e),
            "student_id": student_id,
        }

Above, I created an MCP client to help with connecting to and interacting with the MCP server, which worked like a charm except one thing….it had async functions. To get around this, I made a wrapper for the call. This is necessary since async function calling from Gemini leads to an error.

In [11]:
# Confirm check_eligibility wrapper functionality
check_eligibility(random.choice(students)['student_id'])

'"Student Kevin Pacheco (ID: e73992d2-6535-43d8-9c3e-afb3bb6463c8) has a GPA of 2.76 in Computer Science. They are eligible for the [\'STEM Excellence Award\']. Requirements: [\'Field of Study in STEM\']"'

#### <em>Note: This is the context that will be leveraged by Gemini</em>

In [12]:
# Add the check_eligibility wrapper function
mcp_tools = [check_eligibility]

In [13]:
instruction = """You are a helpful financial aid administrative assistant that can interact 
with a  server for a university. You will take the users questions and query the server
using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use check_availability to see what financial aid options the student is eligible for and 
explain the requirements they met to be eligible."""

# Initialize Gemini with tools + instructions
config = types.GenerateContentConfig(system_instruction=instruction, tools=mcp_tools)
chat = genai_client.chats.create(
    model="models/gemini-1.5-flash-001-tuning",
    config=config
)

# Run model generation for a random student
response = chat.send_message(
    f"What financial options is student {random.choice(students)['student_id']} eligible for?"
)
print("Gemini response:\n")
print(response.text)

Gemini response:

Student Lisa Hensley (ID: 47fcc672-72e7-4ef0-87d4-51e800c3dd31) is eligible for the Humanities Excellence Scholarship. They are eligible because they are majoring in English Literature, which is a field of study in the humanities. 



Great! Let's try another!

In [14]:
response = chat.send_message(
    f"What financial options is student {random.choice(students)['student_id']} eligible for?"
)
print("Gemini response:\n")
print(response.text)

Gemini response:

Student Melissa Peterson (ID: 1ef1b24e-57d3-4bbc-ac6e-e7998526fb98) is eligible for the Behavioral and Social Sciences Grant.  They are eligible because they are majoring in Economics, which is a field of study in the behavioral and social sciences. 



## Student Services Agent Powered by Gemini

Ideally, we deploy an agent that is capable of choosing the tools on its own. The following defines the Student Services Agent that can be fed the model, tools, mcp-client, and instructions to proceed with everything above on its own.

### Define Student Services Agent

In [15]:
class StudentServicesAgent:
    def __init__(self, 
                 api_key: str,
                 model: str,
                 tools: List[Dict[str, Any]], 
                 mcp_client: StudentServicesMCPClient,
                 instructions: str
    ):
        configure(api_key=api_key)
        self.model = GenerativeModel(
            model, 
            tools=tools,
            system_instruction=instructions
        )
        self.chat = self.model.start_chat(history=[])
        self.memory: List[Dict[str, Any]] = []
        self.mcp_client = mcp_client

    async def run_tool(self, name: str, args: dict) -> str:
        try:
            if name == "check_financial_aid_eligibility":
                return await self.mcp_client.check_financial_aid_eligibility(args["student_id"])
            elif name == "fetch_student_profile":
                return await self.mcp_client.fetch_student_profile(args["student_id"])
            elif name == "fetch_students":
                return await self.mcp_client.fetch_students(args.get("limit", 10))
            else:
                return f"[Tool {name}] not implemented."
        except Exception as e:
            return f"[Tool Error] {str(e)}"

    def print_chat_history(self):
        print("Chat History:")
        for i, turn in enumerate(self.memory):
            role = turn.get("role", "unknown")
            if role == "user":
                content = turn["parts"][0]["text"]
                print(f"\nUser: {content}")
            elif role == "model":
                content = turn["parts"][0]["text"]
                print(f"\nModel: {content}")
            elif role == "function":
                name = turn.get("name", "unknown_tool")
                content = turn["parts"][0]["text"]
                print(f"\nTool [{name}] Output:\n{content}")


    async def process_message(self, message: str) -> str:
        self.memory.append({"role": "user", "parts": [{"text": message}]})
        response = self.chat.send_message(message)

        parts = response.candidates[0].content.parts

        # Handle tool calling
        if parts and hasattr(parts[0], "function_call"):
            func_call = parts[0].function_call
            tool_result = await self.run_tool(func_call.name, func_call.args)

            # Store tool output in memory
            self.memory.append({
                "role": "function",
                "name": func_call.name,
                "parts": [{"text": str(tool_result)}]
            })

            # Feed tool result back to Gemini
            final_response = self.chat.send_message(str(tool_result))
            self.memory.append({
                "role": "model",
                "parts": [{"text": final_response.text}]
            })

            return final_response.text

        # Normal message (no function call)
        self.memory.append({
            "role": "model",
            "parts": [{"text": response.text}]
        })
        return response.text


### Define Tool Schema

In [16]:
tools = [{
    "function_declarations": [
        {
            "name": "check_financial_aid_eligibility",
            "description": "Check if a student is eligible for financial aid",
            "parameters": {
                "type": "object",
                "properties": {
                    "student_id": {"type": "string"}
                },
                "required": ["student_id"]
            }
        },
        {
            "name": "fetch_student_profile",
            "description": "Get detailed info for a student",
            "parameters": {
                "type": "object",
                "properties": {
                    "student_id": {"type": "string"}
                },
                "required": ["student_id"]
            }
        },
        {
            "name": "fetch_students",
            "description": "List students with a limit",
            "parameters": {
                "type": "object",
                "properties": {
                    "limit": {"type": "integer"}
                }
            }
        }
    ]
}]

### Put Agent to Work

In [17]:
# Instruction Set
agent_instructions = """You are a helpful financial aid administrative assistant that can interact 
with a  server for a university. You will take the users questions and query the server
using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use the tools available to you to see what financial aid options the student is eligible for and 
explain the requirements they met to be eligible. Use the other tools as needed to answer the user's question.
If the tools are not enough to answer the question, inform the user that you cannot answer them at this time.
"""

# Initialize Agent
agent = StudentServicesAgent(
    api_key=GOOGLE_API_KEY, 
    model="models/gemini-1.5-flash-001-tuning", 
    mcp_client=mcp_client, 
    tools=tools,
    instructions=agent_instructions
)

# Test with Student 4
await agent.process_message(f"Can you check if student f{students[4]} is eligible for financial aid?")


'Yes, Kevin Pacheco is eligible for financial aid.  They meet the requirements for the STEM Excellence Award because they are majoring in Computer Science. \n'

Looking promising, let's change gears on the agent with a follow-up

In [18]:
await agent.process_message("Fascinating...can I see their profile information?")

'Kevin Pacheco is a senior in the B.S. in Cybersecurity program. They are majoring in Computer Science and have a GPA of 2.76. They are enrolled in the following courses: HIST300, PHIL101, CS101, ENG202, and MATH150.  He is eligible for need-based aid. \n'

Ok, great. Full profile was provided for the student.

In [19]:
await agent.process_message(
    f"Thank you. Can I see financial aid eligibility details for student with id {random.choice(students)}?"
)

'Allen Robinson is eligible for the STEM Excellence Award, which is available to students in STEM fields.  Nursing is considered a STEM field, so they meet the requirements for this award. \n'

Now for the real test! Can it handle the scenario where the the information is simply not available to it. I never provided a tool for the university details, nor did I ever state it in the chat or instructions. Let's see if it provides something silly.

In [20]:
await agent.process_message(
    "What is the university's name?"
)

"I apologize, but I do not have access to information about the university's name. The tools I have available do not provide this type of information. \n"

We love to see it!

### Get Chat History

In [21]:
agent.print_chat_history()

Chat History:

User: Can you check if student f{'student_id': 'e73992d2-6535-43d8-9c3e-afb3bb6463c8', 'name': 'Kevin Pacheco', 'courses': "['HIST300', 'PHIL101', 'CS101', 'ENG202', 'MATH150']", 'is_need_based_qualified': True, 'enrollment_status': 'enrolled', 'major': 'Computer Science', 'program': 'B.S. in Cybersecurity', 'year': 'Senior', 'gpa': 2.76} is eligible for financial aid?

Tool [check_financial_aid_eligibility] Output:
[TextContent(type='text', text='"Student Kevin Pacheco (ID: e73992d2-6535-43d8-9c3e-afb3bb6463c8) has a GPA of 2.76 in Computer Science. They are eligible for the [\'STEM Excellence Award\']. Requirements: [\'Field of Study in STEM\']"', annotations=None)]

Model: Yes, Kevin Pacheco is eligible for financial aid.  They meet the requirements for the STEM Excellence Award because they are majoring in Computer Science. 


User: Fascinating...can I see their profile information?

Tool [fetch_student_profile] Output:
[TextContent(type='text', text='"Student Kevin 

While the built-in Gemini chat history capabilities were certainly helpful the first time around, I required the knowledge of what tool was being called and the output to assist with visually confirming the flow.

## Agent Evaluation

Following the example from module 1, we will be using an evaluator agent to see how the agent's responses are holding up.

In [22]:
TOOL_AGENT_EVAL_PROMPT = """\
# Instruction
You are an expert evaluator. Your task is to assess the quality of an AI agent's response when using external tools (e.g., APIs, database queries).
You will be provided with:
- The user's input
- The tool output(s) retrieved to help answer the input
- The final AI response to the user

You should determine if the response:
1. Correctly follows the user's instruction
2. Accurately uses the tool outputs (i.e., no hallucination or misrepresentation)
3. Is clear, concise, and well-written

# Evaluation Criteria
## Instruction Following
Did the agent follow the user's prompt and intent?

## Groundedness (Tool Usage)
Is the agent’s response consistent with the output(s) from the tool(s)?
Does it avoid inventing facts not in the tool output?

## Clarity & Helpfulness
Is the response easy to read and useful for the user?

# Rating Rubric
5: (Very good). Fully follows instructions, grounded in tool output, clear and helpful.
4: (Good). Mostly follows instructions and groundedness, minor phrasing issues.
3: (OK). Acceptable, but has some flaws in phrasing, tool use, or instruction interpretation.
2: (Bad). Follows instructions but misuses tool results or lacks clarity.
1: (Very bad). Hallucinates or ignores tool output, fails instructions.

# Evaluation Steps
STEP 1: Read the user input, tool output(s), and AI response.
STEP 2: Evaluate against the criteria.
STEP 3: Assign a rating based on the rubric, and explain your reasoning.

# User Input
{user_prompt}

# Tool Output(s)
{tool_outputs}

# AI Response
{response}
"""

In [23]:
class SummaryRating(enum.Enum):
  VERY_GOOD = '5'
  GOOD = '4'
  OK = '3'
  BAD = '2'
  VERY_BAD = '1'

def eval_tool_response(user_prompt, tool_outputs, ai_response):
    """Evaluate a tool-using agent's response for instruction, grounding, and quality."""

    chat = genai_client.chats.create(model="gemini-1.5-pro")  # or flash, depending on speed vs quality

    eval_prompt = TOOL_AGENT_EVAL_PROMPT.format(
        user_prompt=user_prompt,
        tool_outputs=tool_outputs,
        response=ai_response
    )

    # Get full explanation
    response = chat.send_message(eval_prompt)
    verbose_eval = response.text

    # Get structured score
    structured_output_config = types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=SummaryRating,
    )
    response = chat.send_message(
        message="Convert the final score only into one of: 5, 4, 3, 2, 1.",
        config=structured_output_config,
    )
    structured_eval = response.parsed

    return verbose_eval, structured_eval


#### Preview Test Case

In [24]:
# Run against student 1
check_eligibility(students[1]['student_id'])

'"Student Megan Mcclain (ID: d777f2b6-906f-4703-a5f3-efac47766ac0) has a GPA of 2.84 in Nursing. They are eligible for the [\'STEM Excellence Award\']. Requirements: [\'Field of Study in STEM\']"'

### Run Test Case and Eval

In [25]:
user_prompt = f"Check if student {students[1]} is eligible for financial aid."
tool_outputs = check_eligibility(students[1]['student_id'])
ai_response = await agent.process_message(
    user_prompt
)

ai_response

'Megan Mcclain is eligible for the STEM Excellence Award, which is available to students in STEM fields. Nursing is considered a STEM field, so she meets the requirements for this award. \n'

In [26]:
from IPython.display import Markdown

text_eval, score = eval_tool_response(user_prompt, tool_outputs, ai_response)
Markdown(text_eval)

STEP 1: Read the user input, tool output(s), and AI response. (Completed)

STEP 2: Evaluate against the criteria.

* **Instruction Following:** The AI successfully determines and communicates the student's eligibility for financial aid, specifically mentioning the STEM Excellence Award.
* **Groundedness (Tool Usage):** The AI's response is perfectly grounded in the tool output. It correctly extracts the student's name, eligibility for the STEM Excellence Award, and the award's requirement of being in a STEM field.  It correctly identifies Nursing as being considered a STEM field (according to the tool).
* **Clarity & Helpfulness:** The response is clear, concise, and helpful. It directly answers the user's question and provides the relevant context.

STEP 3: Assign a rating based on the rubric, and explain your reasoning.

Rating: **5 (Very good)**

Reasoning: The AI's response perfectly fulfills the user's request. It correctly interprets the tool's output, extracts the relevant information, and presents it in a clear and concise manner.  There are no hallucinations or inaccuracies. The response demonstrates excellent instruction following, groundedness, and clarity.


## Summary

All in all, this notebook presented usage of Gemini with few-shot examples, structured output, and function calling. An agent was built to combine functionality towards the end, providing a single-call setup and the chat history was provided to present user<>model<>tool call chain. The agent was then tested against using an evaluator.

This was honestly fun to put together. Learned a lot in the process! Below is the code for the MCP for those interested.
  
MCP GitHub Repo: <a href="https://github.com/EdyVision/student-services-mcp">Student Services MCP</a>

Thank you to Google and Kaggle for running this course. It was a lot of fun! Until next time…. <em>Happy Hacking!</em>